<a href="https://colab.research.google.com/github/jsdysw/dive-into-deep-learning/blob/master/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import random
import torch
from torch.utils import data
from torch import nn

# Generating Data

In [69]:
# generate y = Xw + b + noise
def synthetic_data(w, b, num_examples):
  # (numexample * len(w))
  X = torch.normal(0,1, (num_examples, len(w)))

  # (numexample * 1)
  y = torch.matmul(X, w) + b

  y += torch.normal(0, 0.01, y.shape)
  return X, y.reshape((-1, 1))

In [70]:
true_w = torch.tensor(([2, -3.4]))
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print("features:", features[0], '\nlabel:', labels[0])

features: tensor([1.0835, 1.3590]) 
label: tensor([1.7477])


# Reading Data (minibatches)
1. load dataset to DataLoader
2. Each iteration returns features/label
3. if shuffle=True, after iterating all batches, it shuffles data (suffle data every epoch)

In [71]:
# Construct a PyTorch data iterator
def load_array(data_arrays, batch_size, is_train=True):
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [72]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [73]:
train_features, train_labels = next(iter(data_iter))
print("features: ", train_features, "\nlabels: ", train_labels)

features:  tensor([[ 1.1135, -0.2616],
        [ 0.1941,  0.8168],
        [-0.1170,  0.1367],
        [ 0.9456,  0.7100],
        [ 1.4620, -0.2690],
        [-0.1388,  0.2234],
        [-1.0058, -0.7264],
        [-1.3819, -1.2641],
        [ 1.1151,  0.8816],
        [-0.0669,  0.6446]]) 
labels:  tensor([[7.3122],
        [1.8084],
        [3.5297],
        [3.6855],
        [8.0375],
        [3.1586],
        [4.6536],
        [5.7443],
        [3.4328],
        [1.8684]])


# Defining Model

In [74]:
net = nn.Sequential(nn.Linear(2,1))

# Initializing Model Parameters

In [75]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

# Defining the loss function

In [76]:
loss = nn.MSELoss()

# Defining the optimization algorithm


In [77]:
trainer = torch.optim.SGD(net.parameters(), lr= 0.03)

# Training

In [78]:
num_epochs = 3
for epoch in range(num_epochs):
  for X, y in data_iter:
    l = loss(net(X), y)
    #  explicitly set the gradients to zero before starting backpropragation
    trainer.zero_grad()
    l.backward()
    # update model parameters
    trainer.step()
  l = loss(net(features), labels)
  print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000173
epoch 2, loss 0.000105
epoch 3, loss 0.000105


In [79]:
w = net[0].weight.data
print('error in estimating w : ', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b : ', true_b - b)

error in estimating w :  tensor([0.0005, 0.0002])
error in estimating b :  tensor([-0.0011])
